In [1]:
from mysql.connector import connect
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import json
import pandas as pd
import datetime

from ssl import create_default_context, CERT_NONE
from elasticsearch import Elasticsearch

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context, CERT_NONE
import json
import pandas as pd
import datetime
import logging
import re

# Alphabet generator

In [2]:
# use the string module
import string
alphabet_list = list(string.ascii_lowercase)

print(alphabet_list)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# ES connection settup

In [4]:
def create_es_connection():
    # Load certificate path and credentials from environment variables
    cafile_path = os.getenv('ES_CAFILE_PATH', r"D:\markding_git\big-data-ai-integration-platform\search_system\es_certs\http_ca.crt")
    es_host = os.getenv('ES_HOST', 'http://localhost:9200')
    es_user = os.getenv('ES_USER', 'elastic')
    es_password = os.getenv('ES_PASSWORD', 'gAcstb8v-lFCVzCBC__a')
    # uWzW*VOzBZknR-Jt3T7i
    context = create_default_context(cafile=cafile_path)
    # context.check_hostname = True  # Ensure hostname is checked
    # context.verify_mode = CERT_REQUIRED  # Ensure the certificate is verified

    context.check_hostname = False
    context.verify_mode = CERT_NONE
    
    es = Elasticsearch(
                [es_host],
                http_auth=(es_user, es_password),
                verify_certs=False,
                )
    return es

es = create_es_connection()
print(es.info())

es_health = es.health_report()
print(es_health['status'])
indicators = es_health['indicators']
[f"status_of_[{i}] : {indicators[i]['status']}" for i in indicators.keys()]

{'name': '44b52d9c8e08', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0XOBCloyQGW6WDdj1nbg3Q', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
yellow


['status_of_[master_is_stable] : green',
 'status_of_[repository_integrity] : green',
 'status_of_[disk] : green',
 'status_of_[shards_capacity] : green',
 'status_of_[shards_availability] : yellow',
 'status_of_[slm] : green',
 'status_of_[ilm] : green']

In [5]:
def es_mapping(df, column_types_dict):
    """
    Generate Elasticsearch mapping based on the DataFrame and a column-to-Python type dictionary.

    Args:
        df (pandas.DataFrame): The DataFrame to be indexed.
        column_types_dict (dict): Dictionary mapping columns to Python types.

    Returns:
        dict: Elasticsearch mapping dictionary.
    """

    # Define a mapping from Python types to Elasticsearch types
    type_conversion = {
        'object': 'keyword',  # Typically strings in Pandas
        'int64': 'integer',
        'int32': 'integer',
        'float64': 'float',
        'float32': 'float',
        'bool': 'boolean',
        'datetime64[ns]': 'date',
        'timedelta[ns]': 'long',
        'category': 'keyword',
    }

    # Generate the Elasticsearch mapping using the provided column types
    mapping = {
        "mappings": {
            "properties": {
                col: {
                    "type": type_conversion.get(column_types_dict[col], "text"),  # Default to "text"
                    "ignore_malformed": True if type_conversion.get(column_types_dict[col]) in ["integer", "float"] else False
                }
                for col in df.columns
            }
        }
    }

    logger.info(f"Mapping: {mapping}")
    return mapping

In [6]:
# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')

con = mysql_connection_whole_corp()
cursor = con.cursor()

cursor.execute('select * from whole_corp limit 100')
result = cursor.fetchall()

df = pd.DataFrame(result, columns= cursor.column_names)


In [18]:
for _, i in df.iterrows():
    print(i['資本額'] == None)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [28]:
es.search(index='template_list', body={"query": {"match_all": {}}})

ObjectApiResponse({'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'template_list', '_id': 'rRSBDZEBepgBNL0BoP0w', '_score': 1.0, '_source': {'email': 'sample1@email.com', '手機': 933123456, '中文姓名': '王小明', '公司名稱': '英丰寶', '部門': 'IT', '職稱': '工程師', '電話區碼': 2, '電話': 26551111, '分機': 866, '統一編號': 89594201, '產業別': '軟體資訊', '生日': '1990-01-01', '性別': '男', '郵遞區號': 106, '地址': '台北市大安區敦化南路二段76號7樓', '資料來源': '英丰寶'}}, {'_index': 'template_list', '_id': 'rhSBDZEBepgBNL0BoP0w', '_score': 1.0, '_source': {'email': 'sample2@email.com', '手機': 933123457, '中文姓名': '李小君', '公司名稱': '英丰寶', '部門': '行銷', '職稱': '專員', '電話區碼': 3, '電話': 26551112, '分機': 862, '統一編號': 89594201, '產業別': '軟體資訊', '生日': '1990-01-02', '性別': '女', '郵遞區號': 106, '地址': '台北市大安區敦化南路二段76號7樓', '資料來源': '英丰寶'}}, {'_index': 'template_list', '_id': 'rxSBDZEBepgBNL0BoP0w', '_score': 1.0, '_source': {'email': 'sample3@email.c

# Mapping for each tables

# Create index with mapping in ES

In [19]:
if __name__ == '__main__':
    es = create_es_connection()
    index_name = 'whole_corp'
    if es.indices.exists(index = index_name):
        if str(input('Index {index_name} already exists. Delete? [y/n]').lower()) == 'y':
            es.indices.delete(index=index_name)
        else:
            print('count:', es.count(index = index_name))

    response = es.indices.create(index=index_name, body=corp_mapping())
    
    # Checking mapping in each index
    print(f"/n=============response, es.indices.get_mapping(index={index_name})=============")
    print(response, es.indices.get_mapping(index=index_name))
    

/n=============response, es.indices.get_mapping(index=whole_corp)=============
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'whole_corp'} {'whole_corp': {'mappings': {'properties': {'公司名稱': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}, '區域名稱': {'type': 'keyword'}, '官網': {'type': 'keyword', 'index': False}, '營業項目及代碼表': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}, '登記地址': {'type': 'text'}, '統一編號': {'type': 'keyword'}, '縣市區域': {'type': 'keyword'}, '縣市名稱': {'type': 'keyword'}, '負責人': {'type': 'text', 'fields': {'keyword': {'type': 'keyword'}}}, '資本額': {'type': 'double'}, '電話': {'type': 'keyword'}, '類別_全': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}, 'analyzer': 'traditional_chinese_analyzer'}}}}}


# Adjusted mapping

In [19]:
'''-----------------------------Create mapping for user----------------------------------'''
def corp_mapping():
    # Define the settings for the custom analyzer
    body = {
        "properties": {
          "cluster": {
            "type": "keyword"
          },
          "vector": {
            "type": "dense_vector",
            "dims": 100    # // must match your model output
          }
        }
      }

    return body


In [20]:
es.indices.put_mapping(index='whole_corp', body=corp_mapping())

ObjectApiResponse({'acknowledged': True})

# Import data

In [ ]:
es = create_es_connection()
for index_name in ['whole_corp']:
    try:
        print(f'index name : [{index_name}]',es.count(index=index_name.lower()))
    except Exception as e:
        print(f'\n==============no values in index name [{index_name}]================')
es.close()

index name : [whole_corp] {'count': 1144129, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


In [ ]:
es.indices.delete(index='whole_corp')

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [whole_corp/ThaN78eZT9KDRpBMFHo8fw] already exists')

# Data import to ES from mysql

In [ ]:
'''=============================================['userESG', 'lifeCircleESG', 'courseESG']==============================================='''
# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')


def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    if isinstance(obj, bytearray):
        return obj.decode('utf-8')
    raise TypeError ("Type %s not serializable" % type(obj))
    

# Generator function to fetch data in batches from MySQL
def fetch_data(cursor, batch_size=1000):
    while True:
        rows = cursor.fetchmany(batch_size)
        if not rows:
            break
        # Convert each row to JSON serializable format
        cleaned_rows = []
        for row in rows:
            cleaned_row = {k: json_serial(v) if isinstance(v, (datetime.datetime, bytearray)) else 0.0 if (k == '資本額') and (v == '\\N') else v for k, v in row.items()}
            cleaned_rows.append(cleaned_row)
        yield cleaned_rows
    

def elastic_import(es, actions):
    response, errors  = helpers.bulk(es, actions, raise_on_error=False)
    if errors:
        print(f'==================================errors: ===============================\n{errors}')
        
    
# Main function to control the flow of the script
def main(table_name, database = None):
    # Connect to MySQL
    
    mysql_conn = mysql_connection_whole_corp() 
        
    cursor = mysql_conn.cursor(dictionary=True)
    
    # Define the SQL query and execute it
    sql_query = f"SELECT * FROM {table_name}"
    cursor.execute(sql_query)
    
    # Connect to Elasticsearch
    es = create_es_connection()

    # Define the Elasticsearch index where the data will be stored
    es_index = table_name.lower()
    
    # Fetch data from MySQL and prepare it for Elasticsearch
#     if es.indices.exists(index = table_name):
#         return 'the index already exists'
    
    for batch in fetch_data(cursor):
        actions = [
            {
                "_index": es_index,
                "_source": row,
            } for row in batch
        ]
        
        # Bulk index the data in Elasticsearch
        elastic_import(es, actions)

    # Cleanup
    cursor.close()
    mysql_conn.close()
    return actions
    
    
if __name__ == "__main__":
    
    #Define paramenter
    database = 'whole_corp'
    table_name = 'whole_corp'
    es_data = main(table_name,database)


==================================errors: ===============================
[{'index': {'_index': 'whole_corp', '_id': '1CmiJ5kBifl7WjgxL4wN', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:169] failed to parse field [資本額] of type [double] in document with id '1CmiJ5kBifl7WjgxL4wN'. Preview of field's value: ' '", 'caused_by': {'type': 'number_format_exception', 'reason': 'empty String'}}}}]
==================================errors: ===============================
[{'index': {'_index': 'whole_corp', '_id': 'wCmiJ5kBifl7WjgxMJT-', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:157] failed to parse field [資本額] of type [double] in document with id 'wCmiJ5kBifl7WjgxMJT-'. Preview of field's value: ' '", 'caused_by': {'type': 'number_format_exception', 'reason': 'empty String'}}}}, {'index': {'_index': 'whole_corp', '_id': 'TSmiJ5kBifl7WjgxMZU6', 'status': 400, 'error': {'type': 'document_parsing_exception', 'reason': "[1:175] failed

KeyboardInterrupt: 

# Update index with specific field